<a href="https://colab.research.google.com/github/blythecarr/NLP_E-CommersRatingAndReview/blob/main/STI202303367_WindyPangestuti_NLP_KlasifikasiSentimenE_CommerceRatingAndReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import library

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

2. Load dataset

In [9]:
df = pd.read_csv("/content/raw_review_googleplay.csv")

3. Tampilkan kolom yang tersedia

In [10]:
print("Kolom dalam dataset:", df.columns)

Kolom dalam dataset: Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')


4. Tampilkan 5 data pertama

In [11]:
print(df[['content', 'score']].head())

                                             content  score
0                   bakalan langganan belanja disini    5.0
1  Saya dengar ada promo pengguna baru diskon sam...    3.0
2                                             mantap    5.0
3                                      banyak diskon    5.0
4  Kurang Transparan TAGIHAN TOKOPEDIA CARD, apa ...    2.0


5. Buat label sentimen berdasarkan skor rating

In [12]:
def label_sentimen(score):
    if score >= 4:
        return 1  # Positif
    elif score <= 2:
        return 0  # Negatif
    else:
        return None  # Netral

df['label'] = df['score'].apply(label_sentimen)
df = df.dropna(subset=['label']).copy()  # Hapus data netral
df['label'] = df['label'].astype(int)

/tmp/ipython-input-12-3882227936.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


6. Tokenisasi teks ulasan

In [13]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['content'])

sequences = tokenizer.texts_to_sequences(df['content'])
padded = pad_sequences(sequences, maxlen=100, padding='post')

In [36]:
print("Token hasil tokenisasi:", ulasan_seq)


Token hasil tokenisasi: [[40, 34, 5, 12, 587]]


7. Split data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(padded, df['label'], test_size=0.2, random_state=42)

8. Bangun model LSTM

In [40]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [41]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [42]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

9. Training model

In [43]:
model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop]
)


Epoch 1/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 31s 108ms/step - accuracy: 0.5696 - loss: 0.6841 - val_accuracy: 0.6107 - val_loss: 0.6706
Epoch 2/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 40s 103ms/step - accuracy: 0.5834 - loss: 0.6800 - val_accuracy: 0.6107 - val_loss: 0.6743
Epoch 3/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 41s 102ms/step - accuracy: 0.6029 - loss: 0.6521 - val_accuracy: 0.8305 - val_loss: 0.4424
Epoch 4/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 41s 101ms/step - accuracy: 0.8255 - loss: 0.4665 - val_accuracy: 0.8315 - val_loss: 0.4398
Epoch 5/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 41s 101ms/step - accuracy: 0.8192 - loss: 0.4695 - val_accuracy: 0.8320 - val_loss: 0.4521
Epoch 6/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 42s 104ms/step - accuracy: 0.8223 - loss: 0.4735 - val_accuracy: 0.8331 - val_loss: 0.4378
Epoch 7/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 40s 101ms/step - accuracy: 0.8219 - loss: 0.4723 - val_accuracy: 0.8341 - val_loss: 0.4367
Epoch 8/15
239/239 ━━━━━━━━━━━━━━━━━━━━ 41s 101ms/step - accuracy: 0.8235 - loss: 0

10. Evaluasi model

In [44]:
loss, acc = model.evaluate(X_test, y_test)
print(f'\n✅ Akurasi Testing: {acc * 100:.2f}%')

75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8181 - loss: 0.4610

✅ Akurasi Testing: 81.70%


11. uji coba

In [53]:
print("\n🔍 Uji Coba Prediksi Ulasan Baru")

ulasan_baru = input("Masukkan ulasan Anda: ")
ulasan_bersih = bersihkan_teks(ulasan_baru)

ulasan_seq = tokenizer.texts_to_sequences([ulasan_bersih])
print("Token hasil tokenisasi:", ulasan_seq)  # Debug

if len(ulasan_seq[0]) == 0:
    print("⚠️ Kata-kata dalam ulasan tidak dikenali oleh model.")
else:
    ulasan_pad = pad_sequences(ulasan_seq, maxlen=100, padding='post')
    prediksi = model.predict(ulasan_pad)
    skor = prediksi[0][0]
    print(f"\n📊 Skor prediksi: {skor:.4f}")

    if skor > 0.6:
        print("✅ Hasil Prediksi: Positif")
    elif skor < 0.4:
        print("❌ Hasil Prediksi: Negatif")
    else:
        print("❔ Hasil Prediksi: Netral/Ragu-ragu")


🔍 Uji Coba Prediksi Ulasan Baru
Masukkan ulasan Anda: barang bagus
Token hasil tokenisasi: [[12, 22]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

📊 Skor prediksi: 0.8768
✅ Hasil Prediksi: Positif
